In [1]:
# importing libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

2024-10-13 07:46:50.792381: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 07:46:53.382258: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 07:46:55.296547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 07:46:57.286672: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 07:46:57.873322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 07:47:00.812974: I tensorflow/core/platform/cpu_feature_gu

In [3]:
# loading datasets
train_dir = 'datasets/students/train'
test_dir = 'datasets/students/test'

# preparing 
img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    zoom_range = 0.2,
    rotation_range = 0.3,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1. / 255
)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)



Found 1071 images belonging to 2 classes.
Found 327 images belonging to 2 classes.


In [13]:
# creating a model
model = Sequential(
    [
        Input(shape=(img_width, img_height, 3)),
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

# compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │     1,605,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,846,977 (7.05 MB)

 Trainable params: 1,846,977 (7.05 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# training the model
checkpoint = ModelCheckpoint('best_model_st.keras', monitor='val_accuracy',  mode='max', save_best_only=True, verbose=1)
reduce_lr = reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.001)
history = model.fit(train_generator, epochs=5, validation_data=test_generator, callbacks=[checkpoint, reduce_lr])

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9723 - loss: 0.0610
Epoch 1: val_accuracy improved from -inf to 0.42017, saving model to best_model_st.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 71s 9s/step - accuracy: 0.9726 - loss: 0.0607 - val_accuracy: 0.4202 - val_loss: 3.4676 - learning_rate: 0.0010
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9905 - loss: 0.0514
Epoch 2: val_accuracy improved from 0.42017 to 0.44538, saving model to best_model_st.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 83s 10s/step - accuracy: 0.9897 - loss: 0.0529 - val_accuracy: 0.4454 - val_loss: 6.3154 - learning_rate: 0.0010
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - accuracy: 0.9671 - loss: 0.1434 
Epoch 3: val_accuracy did not improve from 0.44538
8/8 ━━━━━━━━━━━━━━━━━━━━ 174s 21s/step - accuracy: 0.9661 - loss: 0.1436 - val_accuracy: 0.3950 - val_loss: 3.2921 - learning_rate: 0.0010
Epoch 4/5
2/8 ━━━━━━━━━━━━━━━━━━━━ 5:55 59s/step - accuracy: 0.8359 - loss: 0.2719

In [15]:
# evaluating the model
loss, accuracy = model.evaluate(test_generator)
print(f"Loss:{loss:.4f}, Accuracy:{accuracy*100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.4697 - loss: 2.5169
Loss:2.5944, Accuracy:47.90%


In [5]:
#**********************************************************************
from tensorflow.keras.applications import VGG16


In [4]:

# Load the VGG16 model pre-trained on ImageNet without the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 35s 1us/step


In [6]:
for layer in base_model.layers:
    layer.trainable = False  # Freezes the layer


In [9]:
from tensorflow.keras import layers, models

# Create a new model
model = models.Sequential()

# Add the pre-trained base model
model.add(base_model)

# Add custom layers
model.add(layers.Flatten())  # Flatten the output of the base model
model.add(layers.Dense(512, activation='relu'))  # Fully connected layer
model.add(layers.Dropout(0.5))  # Dropout to prevent overfitting
model.add(layers.Dense(10, activation='softmax'))  # Output layer for 10 classes


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    './/datasets/students/train/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    './/datasets/students/test/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1071 images belonging to 2 classes.


Found 327 images belonging to 2 classes.


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Number of batches per epoch
    epochs=10,  # Number of epochs to train
    validation_data=validation_generator,  # Validation data generator
    validation_steps=50  # Number of batches to validate
)


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 10)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Test accuracy:', test_acc)


In [ ]:
model.save('my_transfer_learning_model.h5')


In [ ]:
#**********************************************************************